In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers,Model,utils
import os
import datetime
import time
import re
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv("rare_event_detection.csv")

In [3]:
data.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127998,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161567,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0


In [4]:
data.describe()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
count,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,...,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000,18398.000000
mean,0.006740,0.011824,0.157986,0.569300,-9.958345,0.006518,2.387533,0.001647,-0.004125,-0.003056,...,0.380519,0.360246,0.173708,2.379154,9.234953,0.233493,-0.001861,-0.061522,0.001258,0.001033
std,0.081822,0.742875,4.939762,5.937178,131.033712,0.634054,37.104012,0.108870,0.075460,0.156047,...,6.211598,14.174273,3.029516,67.940694,81.274103,2.326838,0.048732,10.394085,0.004721,0.032120
min,0.000000,-3.787279,-17.316550,-18.198509,-322.781610,-1.623988,-279.408440,-0.429273,-0.451141,-0.120087,...,-187.943440,-1817.595500,-8.210370,-230.574030,-269.039500,-12.640370,-0.149790,-100.810500,-0.012229,0.000000
25%,0.000000,-0.405681,-2.158235,-3.537054,-111.378372,-0.446787,-24.345268,-0.058520,-0.051043,-0.059966,...,-3.672684,-1.928166,0.487780,-40.050046,-45.519149,-1.598804,0.000470,0.295023,-0.001805,0.000000
50%,0.000000,0.128245,-0.075505,-0.190683,-14.881585,-0.120745,10.528435,-0.009339,-0.000993,-0.030057,...,0.294846,0.143612,0.702299,17.471317,1.438806,0.085826,0.012888,0.734591,0.000710,0.000000
75%,0.000000,0.421222,2.319297,3.421223,92.199134,0.325152,32.172974,0.060515,0.038986,0.001990,...,5.109543,3.230770,2.675751,44.093387,63.209681,2.222118,0.020991,1.266506,0.004087,0.000000
max,1.000000,3.054156,16.742105,15.900116,334.694098,4.239385,96.060768,1.705590,0.788826,4.060033,...,14.180588,11.148006,6.637265,287.252017,252.147455,6.922008,0.067249,6.985460,0.020510,1.000000


In [5]:
data['y'].value_counts()

0    18274
1      124
Name: y, dtype: int64

We can see that given 124 anomalies out of 18398 samples, the event that we are trying to detect is very rare ~0.674%.

According to the original paper, the features/columns of the dataset represents continous instrumentation data, likely analog force signals measured via loadcells, accelerometers, encoders, drive current loads etc. While x28 is categorical and x61 is binary. In the next few cells we will separate the continuous and categorical features in order to encode them more effectively.

In [6]:
ContinuousData = data.iloc[:,2:-1].drop(columns = ["x28"])

The rates of change overtime of the features are likely to contribute more to the web breaks and failures that we are trying to detect. In order to capture that information we'll compute the rates of change via the first order difference and concatenate them as features to our existing dataset.

A general rule of thumb is that for complex feature sets, where the features are not independent of eachother and redundancies may exist, the number of samples $N$ in the dataset must be larger than $\sqrt{N}$ in order to avoid the "curse of dimensionality". The addition of first-order derivatives for the continuous features will take us to 120 features, and $120^2=14400 < 18274$. So we can cautiously conclude that we have enough data to avoid needing too many parameters in our model and risking overfitting. (we actually have 127 features after we one-hot encode the categorical data, but we're still ok)

First order central difference is given by:
\begin{align}
\\
f'(x) \approx \frac{f(x+\frac{1}{2}h)-f(x-\frac{1}{2}h)}{h}\\
\end{align}

Central difference is used since it has the most accurate approximation. The $h$ we will use is 4 mins.
It is also convenient to ensure that the indexing of the data doesn't change regardless of the fact that the derivatives can't be computed for the first and last terms in the dataset.

In [7]:
def FirstOrderCentralDifference(data):
    data_backward = np.array(data.copy().iloc[:-2,:])
    data_forward = np.array(data.copy().iloc[2::,:])
    
    first_order_features = (data_forward-data_backward)/4.0
    old_names = data.columns.values.tolist()
    new_names = [('d'+name) for name in old_names]
    first_order_features = pd.DataFrame(first_order_features,index = range(1,18397), columns = new_names)
    
    return pd.concat([data.drop([0,18397]),first_order_features],axis = 1)

In [8]:
DataNewFeatures = FirstOrderCentralDifference(ContinuousData)

In [9]:
DataNewFeatures

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,dx51,dx52,dx53,dx54,dx55,dx56,dx57,dx58,dx59,dx60
1,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,-0.059966,-0.038189,...,0.0,0.002136,0.004761,-0.000373,-2.398407,2.122307,0.016022,0.000046,0.221830,0.000004
2,0.363848,-4.681394,-4.353147,14.127998,-0.138636,-17.836632,0.010803,-0.061114,-0.030057,-0.018352,...,0.0,0.002197,0.004700,-0.000373,-0.885254,-0.271904,0.325790,0.000046,0.366094,0.000004
3,0.301590,-4.758934,-4.023612,13.161567,-0.148142,-18.517601,0.002075,-0.061114,-0.019986,-0.008280,...,0.0,0.002197,0.004700,-0.000372,-0.885254,-1.274353,0.016022,0.000037,-0.107511,0.000004
4,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,-0.030057,-0.008280,...,0.0,0.002197,0.004761,-0.000373,-0.068024,0.410110,-0.080109,-0.000024,-0.417061,0.000004
5,0.381253,-4.611746,-4.085072,14.143195,-0.162501,-16.494255,0.000732,-0.061114,-0.030057,-0.008280,...,0.0,0.002197,0.004700,-0.000372,1.374176,0.702812,0.128174,-0.000076,-0.021938,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18392,-0.863607,0.735248,0.266091,133.555731,0.083242,26.922188,-0.139347,0.058823,-0.080108,-0.038189,...,0.0,0.000000,0.000000,0.000462,0.331757,0.407589,-0.160218,-0.000384,0.136722,0.000004
18393,-0.877442,0.786430,0.406426,135.301215,0.112295,26.300392,-0.159185,0.058823,-0.080108,-0.038189,...,0.0,0.000000,0.000000,0.000463,-3.560700,-0.987248,0.138733,0.000200,-0.223778,0.000004
18394,-0.843988,0.633086,0.561918,133.228949,0.141332,25.678597,-0.159185,0.058823,-0.080108,-0.038189,...,0.0,0.000000,0.000000,0.000463,-1.033875,-0.390385,-0.137344,0.000776,-0.068666,0.000004
18395,-0.826547,0.450126,0.334582,134.977973,0.170370,25.056801,-0.159185,0.048752,-0.080108,-0.038189,...,0.0,0.000000,0.000000,0.000463,0.414276,0.000000,0.013870,0.000567,0.303889,0.000004


Now we identify the indices of the anomalies (where y=1), and time shift it by 2 mins by shifting the indices back by 1. 

In [10]:
data.drop([0,18397],inplace = True)
y = data.y
anomalies = y[y==1].index.copy()-1

Now we use the standard scaler from sklearn to fit the continuous dataset without the anomalies, we will then use this scaler to transform the dataset before training.

In [11]:
scaler = StandardScaler().fit(DataNewFeatures.drop(anomalies))
DataNewFeatures.iloc[:,:] = scaler.transform(DataNewFeatures)

Then we take the categorical data x28 and apply one-hot encoding, and re-concatenate the categorical/binary data back onto the dataframe.

In [12]:
pca =PCA(n_components=40)
pca.fit(DataNewFeatures.drop(anomalies))
CompressedData = pd.DataFrame(pca.transform(DataNewFeatures),columns = ['pca_'+str(i) for i in range(1,41)],index = DataNewFeatures.index)

In [13]:
CompressedData

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,...,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,pca_37,pca_38,pca_39,pca_40
1,0.167629,2.600991,-3.226725,1.756417,0.285213,-1.979013,-0.302688,-2.039714,1.109541,0.673106,...,1.477287,-0.821176,0.034368,0.195207,0.694806,-0.269063,-0.232485,0.328578,0.164787,0.281446
2,0.014363,2.742412,-3.392821,1.572611,-0.508399,-2.034450,0.481640,-1.747038,1.158701,0.418357,...,1.215598,-0.314943,-0.231679,0.818819,-0.982866,-0.600509,-0.228666,-0.025240,-0.212320,0.717907
3,-0.071939,2.845502,-3.590585,1.751235,-0.784058,-1.971026,0.558024,-1.770653,0.733993,-0.028914,...,-0.543751,0.379801,0.401707,0.813037,-0.948060,0.354322,-0.657913,0.238481,0.346375,0.189431
4,0.004690,2.868572,-3.472332,1.838193,0.003919,-1.859919,0.093462,-2.026112,0.843449,0.117172,...,-0.165241,-0.130753,-2.057370,-1.463415,0.987264,-0.123735,1.175657,0.245622,0.829409,0.048033
5,0.230351,2.727784,-3.105792,1.796413,0.057416,-2.177032,0.089564,-1.702631,0.805507,0.115606,...,-0.524293,0.180993,-2.130938,-1.459710,0.381737,-0.418009,1.572110,-0.230047,0.288503,0.133784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18392,-0.136243,-1.177658,2.352381,-0.825732,0.306950,-0.572364,-1.183292,1.875247,0.489925,0.043598,...,1.183861,-0.467027,0.898818,1.146940,-0.916532,-0.777103,-0.126197,-0.722377,0.209026,-0.442740
18393,-0.054323,-1.098553,2.099750,-0.939712,-0.460810,-0.551409,-1.245552,1.716765,0.541223,0.006710,...,-0.856869,-0.139845,0.235947,1.248154,-2.447055,-0.197771,-1.184267,-0.077798,0.418340,-0.328965
18394,-0.171015,-1.021022,2.079271,-0.771249,0.012388,-0.447343,-1.421327,1.678810,0.597291,0.154160,...,-1.054152,0.612446,-1.080636,-1.055945,-0.968026,0.804839,0.547078,0.038820,0.449447,-0.283269
18395,-0.045101,-1.130354,2.366625,-0.623990,0.413168,-0.621332,-1.085385,1.972234,0.695140,-0.153862,...,-1.124824,0.097913,-1.648953,-0.972487,-1.158619,0.401118,0.575117,-0.369345,0.771350,0.465586


In [14]:
x28 = pd.get_dummies(data.x28)
x28 = x28.rename(columns = dict(zip(x28.columns.values.tolist(),[('x28_'+str(name)) for name in x28.columns.values.tolist()])))
CompressedData = pd.concat([CompressedData,x28,data.x61],axis = 1)

Here we divide the dataframe into the training data and anomalous data, since we won't be using the anomalous data in the training set. Thus taking advantage of the fact that neural network models can't extrapolate very effectively.

In [15]:
training_data = CompressedData.drop(anomalies)
anomalous_data = CompressedData.iloc[anomalies-1,:]

To preprocess the data we need to ensure that all sequences that would include the anomalies are removed, then we divide the dataset into sequences and return it as a np array.

In [16]:
def PreprocessData(data,seq_len=15):
    dataset = np.array(data,'float32')
    s = set(range(0,len(data)-seq_len))
    for i in anomalies:
        s.difference_update(set(range(i-seq_len+1,i+1)))
    sequences = []
    for i in list(s):
        sequences.append(dataset[i:i+seq_len])
    return np.array(sequences)

The shape of the dataset will ultimately be (samples,sequence length, feature length)

In [17]:
dataset = PreprocessData(training_data)
dataset.shape

(16581, 15, 49)

In [18]:
batched_dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(1000).batch(100,drop_remainder=True)
batched_dataset

<BatchDataset shapes: (100, 15, 49), types: tf.float32>

In [19]:
k=3 # kernel size
latent_dim = 256 # dimension of the latent z
n_filters=128 # initial filter depth for encoder
depth = 3 # depth of dialated TCN
seq_len = 15 # length of the time sequence
feature_dim = 49 # feature dimension for input tensor

In [20]:
class Encoder(layers.Layer):
    def __init__(self,k,latent_dim,n_filters,depth):
        super(Encoder,self).__init__()
        self.depth = depth
        
        self.tcn_block =[
                        layers.Conv1D(n_filters*2**(i),kernel_size = k,strides = 1, 
                        padding = 'causal',dilation_rate = 2**i,activation = 'elu')
                        for i in range(depth)
                        ]
        
        self.dropouts = [layers.Dropout(0.3) for i in range(depth)]
        self.flat = layers.Flatten()
        self.dense = layers.Dense(latent_dim*2)
        
        
    def call(self,inputs,training = True):
        # input dim = (100,15,49)
        x = inputs
        for i in range(self.depth):
            x = self.tcn_block[i](x)
            x = self.dropouts[i](x,training = training) 
        # dim = (100,15,512)
        x = self.flat(x)
        x = self.dense(x)
        # output dim = (100,512)
        return x

In [21]:
class Decoder(layers.Layer):
    def __init__(self,k,latent_dim,n_filters,depth,seq_len,feature_dim):
        super(Decoder,self).__init__()
        self.depth = depth
        self.dense1 = layers.Dense(seq_len*n_filters*2**(depth-1),activation = 'elu')
        self.reshape = layers.Reshape((seq_len,n_filters*2**(depth-1)))
        self.conv_layers =  [
                            layers.Conv1D(n_filters*2**(depth-1-i),
                            kernel_size = k, strides=1, padding = 'SAME', activation = 'elu')
                            for i in range(depth)
                            ]
        self.dropouts = [layers.Dropout(0.3) for i in range(depth)]
        self.generated_sequence = layers.Conv1D(feature_dim, kernel_size = k, strides = 1, padding = 'SAME')

    def call(self,x,training = True):
        # input dim = (100,256)
        x = self.dense1(x)
        # dim = (100,15*256)
        x = self.reshape(x)
        # dim = (100,15,256)
        for i in range(self.depth):
            x = self.conv_layers[i](x)
            x = self.dropouts[i](x,training = training)
        # dim = (100,15,128)
        x = self.generated_sequence(x)
        # output dim = (100,15,49)
        return x

In [22]:
def Reparameterize(mean,logvar,noise):
    z=noise*tf.exp(logvar * 0.5)+mean
    return z

def logPDF(mean,logvar,z):
    logf = -0.5*((z-mean)**2/(tf.math.exp(logvar)+1e-8)+tf.math.log(2*np.pi))
    return logf

def LossFunction(z,mean,logvar,generated_sequence,input_sequence,beta=0.3):
    logpz = tf.reduce_sum(logPDF(0.0,0.0,z),axis = -1)
    logqz = tf.reduce_sum(logPDF(mean,logvar,z),axis = -1)
    logpxz = tf.reduce_sum(tf.keras.losses.MSE(input_sequence,generated_sequence))
    
    total_loss = beta*tf.reduce_mean(-logpz+logqz-logpxz)
    return total_loss

In [23]:
class BVAE(Model):
    def __init__(self,k,latent_dim,n_filters,depth,seq_len,feature_dim):
        super(BVAE,self).__init__()
        self.encoder = Encoder(k,latent_dim,n_filters,depth)
        self.decoder = Decoder(k,latent_dim,n_filters,depth,seq_len,feature_dim)
        self.latent_dim = latent_dim
    
    def call(self,input_sequence,training = True):
        x = self.encoder(input_sequence)
        mean = x[:,:self.latent_dim]
        logvar = x[:,self.latent_dim::]
        noise = tf.random.normal(shape = (100,self.latent_dim))
        z = Reparameterize(mean,logvar,noise)
        generated_sequence = self.decoder(z)
        return z,mean,logvar,generated_sequence

In [24]:
optimizer = tf.keras.optimizers.Adam(1e-4)
model = BVAE(k,latent_dim,n_filters,depth,seq_len,feature_dim)
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)

@tf.function
def train_step(input_sequence):
    with tf.GradientTape() as tape:
        z,mean,logvar,generated_sequence = model(input_sequence)
        loss = LossFunction(z,mean,logvar,generated_sequence,input_sequence)
        
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)

In [25]:
%load_ext tensorboard
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)

In [26]:
start_time = time.time()
for epoch in range(1,201):
    for input_sequence in (batched_dataset):
        train_step(input_sequence)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
    print("training loss epoch {}: {}".format(epoch,train_loss.result()))
    train_loss.reset_states()

training loss epoch 1: nan
training loss epoch 2: nan


KeyboardInterrupt: 

In [26]:
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 9264), started 0:03:00 ago. (Use '!kill 9264' to kill it.)